In [1]:
!pip install scikit-learn

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report
from IPython.display import clear_output

In [3]:
from google.colab import files

def load_dataset():
    if os.path.exists('/content/brain_tumor_mri'):
        return

    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes.'.format(name=fn, length=len(uploaded[fn])))

    !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
    !kaggle datasets download -d jarvisgroot/brain-tumor-classification-mri-images
    !unzip brain-tumor-classification-mri-images.zip
    clear_output()

load_dataset()

In [4]:
data_path = '/content/brain_tumor_mri/new_dataset/bt_images'

training_data_path = os.path.join(data_path, '/content/brain_tumor_mri/new_dataset/training_data.pickle')
labels_path = os.path.join(data_path, '/content/brain_tumor_mri/new_dataset/labels.pickle')
test_data_path = os.path.join(data_path, '/content/test_images-20210704T210303Z-001/test_images')

with open(training_data_path, 'rb') as training_data_file:
    training_data = pickle.load(training_data_file)

with open(labels_path, 'rb') as labels_file:
    labels = pickle.load(labels_file)

In [5]:
# image_path = '/content/brain_tumor_mri/new_dataset/bt_images/1.jpg'
# img = Image.open(image_path)

# width, height = img.size

# print("Image Width:", width)
# print("Image Hieght:", height)

Image Width: 512
Image Hieght: 512


In [6]:
training_data = np.array(training_data)
labels = np.array(labels)

<ipython-input-6-02aadbb422b2>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data = np.array(training_data)


In [7]:
adjusted_labels = labels - 1
unique_labels = np.unique(adjusted_labels)
print(unique_labels)

[0 1 2]


In [8]:
image_data = []

for i in range(3064):
    image_file = f"{i + 1}.jpg"
    image_path = os.path.join(data_path, image_file)
    img = Image.open(image_path)

    img = image.load_img(image_path, target_size=(256, 256))
    image_array = image.img_to_array(img)
    # image_array = np.expand_dims(image_array, axis=0)
    image_array = preprocess_input(image_array)

    image_data.append(image_array)
    del image_array

image_data = np.array(image_data)


In [9]:
train_images, test_images, train_labels, test_labels = train_test_split(image_data, adjusted_labels, test_size=0.1, random_state=42)

In [10]:
del image_data

In [11]:
num_classes = len(np.unique(adjusted_labels))
train_labels_categorical = tf.keras.utils.to_categorical(train_labels, num_classes=num_classes)
test_labels_categorical = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)

NameError: ignored

In [12]:
train_images.shape

(2757, 256, 256, 3)

In [13]:

# pretrained_model = tf.keras.applications.ResNet50V2(
#     include_top=False,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=(224, 224, 3),
#     pooling=None,
#     classes=3,
#     classifier_activation="softmax",
# )

# pretrained_model = tf.keras.applications.ResNet152V2(
#     include_top=False,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=(224, 224, 3),
#     pooling=None,
#     classes=3,
#     classifier_activation="softmax",
# )

pretrained_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(256, 256, 3),
    pooling=None,
    classes=3,
    classifier_activation="softmax",
)a


for layer in pretrained_model.layers:
  layer.trainable=False

model = tf.keras.models.Sequential([
    pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 22328099 (85.17 MB)
Trainable params: 525315 (2.00 MB)
Non-trainable params: 21802784 (83.17 MB)
___________

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

batch_size = 32
epochs = 20

history = model.fit(
    train_images,
    train_labels_categorical,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (test_images, test_labels_categorical)
)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels_categorical, verbose=2)
print("Test accuracy:", test_accuracy)

In [ ]:
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

true_labels = np.argmax(test_labels_categorical, axis=1)

confusion = confusion_matrix(true_labels, predicted_labels)

print("Confusion Matrix:")
print(confusion)

In [ ]:
predictions = model.predict(test_images)

true_labels = test_labels_categorical[:, 1]

auc = roc_auc_score(true_labels, predictions[:, 1])

print("AUC:", auc)
print("Test accuracy:", test_accuracy)

In [ ]:
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracy, label='Train Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
predictions = model.predict(test_images)

true_labels = np.argmax(test_labels_categorical, axis=1)

class_names = ["Pituitary", "Meningioma", "Glioma"]
report = classification_report(
    true_labels,
    np.argmax(predictions, axis=1),
    target_names=class_names
)

print("Classification report:\n", report)

In [ ]:

max_probabilities = np.max(predictions, axis=1)

for i in range(len(max_probabilities)):
  print(f"Test Image {i + 1} - Confidense: {max_probabilities[i]:.4f}")